In [1]:
import torch
import random

from transformers import AutoTokenizer

#加载tokenizer
tokenizer = AutoTokenizer.from_pretrained('google-bert/bert-base-chinese')

tokenizer

BertTokenizerFast(name_or_path='google-bert/bert-base-chinese', vocab_size=21128, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [2]:
from datasets import load_dataset

#加载数据集
dataset = load_dataset(path='lansinuote/ChnSentiCorp')

#过滤句子长度
f = lambda x: len(x['text']) >= 40
dataset = dataset.filter(f)

#移除多余的字段
dataset = dataset.remove_columns(['label'])

dataset, dataset['train'][0]

(DatasetDict({
     train: Dataset({
         features: ['text'],
         num_rows: 8130
     })
     validation: Dataset({
         features: ['text'],
         num_rows: 1032
     })
     test: Dataset({
         features: ['text'],
         num_rows: 1011
     })
 }),
 {'text': '选择珠江花园的原因就是方便，有电动扶梯直接到达海边，周围餐馆、食廊、商场、超市、摊位一应俱全。酒店装修一般，但还算整洁。 泳池在大堂的屋顶，因此很小，不过女儿倒是喜欢。 包的早餐是西式的，还算丰富。 服务吗，一般'})

In [3]:
#定义数据集遍历工具
def collate_fn(data):
    b = len(data)
    text = [i['text'] for i in data]

    #生成前后两段话分别的索引
    s1 = list(range(b))
    s2 = list(range(b))
    random.shuffle(s2)

    #根据索引生成label,表明两句话是否是前后相连的关系
    label = [s1[i] == s2[i] for i in range(b)]

    #取出具体的文字
    s1 = [text[i][0:20] for i in s1]
    s2 = [text[i][20:40] for i in s2]

    #句子对编码
    data = tokenizer(s1,
                     s2,
                     padding=True,
                     truncation=True,
                     max_length=50,
                     return_tensors='pt')

    #设置label
    data['label'] = torch.LongTensor(label)

    return data


loader = torch.utils.data.DataLoader(dataset['train'],
                                     batch_size=4,
                                     shuffle=True,
                                     drop_last=True,
                                     collate_fn=collate_fn)

data = next(iter(loader))

for k, v in data.items():
    print(k, v.shape)

len(loader)

input_ids torch.Size([4, 41])
token_type_ids torch.Size([4, 41])
attention_mask torch.Size([4, 41])
label torch.Size([4])


2032

In [4]:
#查看数据样例
for input_ids, label in zip(data['input_ids'], data['label']):
    print(tokenizer.decode(input_ids))
    print(label)
    print('================')

[CLS] 1 ， 外 壳 晶 莹 剔 透 ， 注 重 外 形 的 人 可 以 选 择 ； [SEP] 0 - 100 的 东 券 ， 节 省 了 100 元 ， 哈 哈 。 [SEP] [PAD] [PAD]
tensor(0)
[CLS] 东 西 很 超 值 ， 买 了 几 件 东 西 ， 用 了 一 个 200 [SEP] 都 很 低 的 时 候 ， 鼠 标 显 示 等 待 ， 就 像 是 死 机 了 [SEP]
tensor(0)
[CLS] 唯 独 不 能 理 解 的 地 方 时 ， cpu 和 内 存 占 用 率 [SEP] 会 觉 得 自 己 过 得 兔 子 一 般 的 生 活 ， 当 然 ， 如 果 [SEP]
tensor(0)
[CLS] 我 是 一 个 无 肉 不 欢 的 人 ， 如 果 每 天 不 吃 点 肉 就 [SEP] 2 ， 显 卡 属 于 中 端 卡 ， 一 般 [UNK] 游 戏 基 本 没 [SEP]
tensor(0)


In [5]:
#定义模型
class Model(torch.nn.Module):

    def __init__(self):
        super().__init__()

        #加载预训练模型
        from transformers import AutoModel
        self.pretrained = AutoModel.from_pretrained(
            'google-bert/bert-base-chinese')

        self.fc = torch.nn.Linear(in_features=768, out_features=2)

    def forward(self, input_ids, attention_mask, token_type_ids, label=None):
        #使用预训练模型抽取数据特征
        with torch.no_grad():
            last_hidden_state = self.pretrained(
                input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids).last_hidden_state

        #只取第0个词的特征做分类,这和bert模型的训练方式有关,此处不展开
        last_hidden_state = last_hidden_state[:, 0]

        #对抽取的特征只取第一个字的结果做分类即可
        out = self.fc(last_hidden_state).softmax(dim=1)

        #计算loss
        loss = None
        if label is not None:
            loss = torch.nn.functional.cross_entropy(out, label)

        return loss, out


model = Model()

model(**data)

(tensor(0.5830, grad_fn=<NllLossBackward0>),
 tensor([[0.6230, 0.3770],
         [0.6774, 0.3226],
         [0.4611, 0.5389],
         [0.7289, 0.2711]], grad_fn=<SoftmaxBackward0>))

In [6]:
#执行训练
def train():
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

    for i, data in enumerate(loader):
        loss, out = model(**data)

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if i % 10 == 0:
            out = out.argmax(dim=1)
            acc = (out == data.label).sum().item() / len(data.label)
            print(i, len(loader), loss.item(), acc)

        if i == 300:
            break


train()

0 2032 0.5397893786430359 1.0
10 2032 0.4697575569152832 1.0
20 2032 0.5538473129272461 0.75
30 2032 0.42610231041908264 1.0
40 2032 0.6893507242202759 0.5
50 2032 0.4129978120326996 1.0
60 2032 0.4780278205871582 0.75
70 2032 0.5568191409111023 0.75
80 2032 0.47240906953811646 0.75
90 2032 0.38953810930252075 1.0
100 2032 0.5210029482841492 1.0
110 2032 0.4725152850151062 1.0
120 2032 0.4108600914478302 1.0
130 2032 0.32916051149368286 1.0
140 2032 0.5040423274040222 0.75
150 2032 0.4608982801437378 0.75
160 2032 0.35494521260261536 1.0
170 2032 0.5046869516372681 0.75
180 2032 0.42960476875305176 1.0
190 2032 0.562688946723938 0.75
200 2032 0.3796224594116211 1.0
210 2032 0.6669230461120605 0.5
220 2032 0.6008779406547546 0.75
230 2032 0.5519516468048096 0.5
240 2032 0.7723520398139954 0.25
250 2032 0.43430188298225403 1.0
260 2032 0.37184998393058777 1.0
270 2032 0.40778613090515137 1.0
280 2032 0.32381296157836914 1.0
290 2032 0.33126550912857056 1.0
300 2032 0.4859747290611267 0.7

In [7]:
#执行测试
def test():
    loader_test = torch.utils.data.DataLoader(dataset['test'],
                                              batch_size=4,
                                              shuffle=True,
                                              drop_last=True,
                                              collate_fn=collate_fn)

    correct = 0
    total = 0
    for i, data in enumerate(loader_test):
        with torch.no_grad():
            _, out = model(**data)

        out = out.argmax(dim=1)
        correct += (out == data.label).sum().item()
        total += len(data.label)

        print(i, len(loader_test), correct / total)

        if i == 5:
            break

    return correct / total


test()

0 252 0.25
1 252 0.625
2 252 0.75
3 252 0.8125
4 252 0.85
5 252 0.875


0.875